# Calculator Sample Client
This sample is focusing purely on building the client using [EZTZ](https://github.com/TezTech/eztz) Javascript library. Contract and all instrumentation code is inherited from [Calculator Dapp sample](calculator_dapp.ipynb), however commentary is skipped to reduce noice. To get more details on instrumentation please consult mentioned sample.

## Sandbox Environment Preparation

In [1]:
# Initialisation of variables to be used later in tutorial
export TEZOS_HOME=~/tezos-dev/tezos
export TEZOS_NODE_URL="127.0.0.1:18731"

# This variable will disable disclaimer about used tezos network. For this guide we will 
# disable it to make command outputs less cluttered. However we strongly discourage to 
# use this setting for your regular work. (source : Tezos Source code reverse engineering)
export TEZOS_CLIENT_UNSAFE_DISABLE_DISCLAIMER=yes
cd $TEZOS_HOME

./src/bin_node/tezos-sandboxed-node.sh \
  1 \
  --cors-header="*" \
  --cors-origin="*" \
  --connections 1 \
  1>tezos_sandbox_log.txt 2>&1 &

#Exporting node pid so we can terminate the sandbox node after the tutorial.
export NODE_PID=$! 
echo $NODE_PID > tezos_sandbox_pid.txt

[1] 49730


In [2]:
#It is required to run following script with the same parameter as the sandbox node was executed (in our case 1)
cd $TEZOS_HOME
eval `./src/bin_client/tezos-init-sandboxed-client.sh 1`

## Tezos address added: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx
## Tezos address added: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN
## Tezos address added: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU
## Tezos address added: tz1b7tUupMgCNw2cCLpKTkSD1NZzB5TkP2sv
## Tezos address added: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
## Tezos address added: tz1TGu6TN5GSez2ndXXeDX6LgUDvLzPLqgYV
## 
## The client is now properly initialized. In the rest of this shell
## session, you might now run `tezos-client` to communicate with a
## tezos node launched with `launch-sandboxed-node 1`. For instance:
## 
##   tezos-client rpc get /chains/main/blocks/head/metadata
## 
## Note: if the current protocol version, as reported by the previous
## command, is "Ps6mwMrF2ER2s51cp9yYpjDcuzQjsc2yAz8bQsRgdaRxw4Fk95H", you
## may have to activate in your "sandboxed network" the same economic
## protocol than used by the alphanet by running:
## 
##   tezos-activate-alpha
## 
## Warning: all the client data will be removed when you cl

In [3]:
tezos-activate-alpha 

tezos-client bake for bootstrap1

Injected BLwhGxwuhomi
Injected block BL7RGw64d1Ak


## EZTZ Client Library Environment
For the sample we will use [EZTZ Javascript Library](https://github.com/TezTech/eztz). You can find documentation [here](https://github.com/TezTech/eztz/wiki/Documentation#eztz.crypto). EZTZ library is able to communicate with Tezos RPC and also provides crypto functions for signing.

At the time of writing this guide, EZTZ is not exported as NPM model. To make it work we need to prepare the project manually and fetch the library directly from github.

Following steps are required when running EZTZ using nodejs. When embedding EZTZ in browser, it is required to embedd [eztz.min.js](https://github.com/TezTech/eztz/raw/master/dist/eztz.min.js) file.


In [4]:
mkdir -p ~/tezos-dev/calculator_dapp_client && cd ~/tezos-dev/calculator_dapp_client

# Fetch of dependencies for 
npm init -y
npm install --save bs58check libsodium-wrappers bip39 xhr2 bignumber.js

wget https://github.com/TezTech/eztz/raw/master/src/main.js -O eztz.js

Wrote to /Users/martinkovacik/tezos-dev/calculator_dapp_client/package.json:

{
  "name": "calculator_dapp_client",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "dependencies": {
    "bignumber.js": "^8.0.2",
    "bip39": "^2.5.0",
    "bs58check": "^2.1.2",
    "libsodium-wrappers": "^0.7.4",
    "xhr2": "^0.1.4"
  },
  "devDependencies": {},
  "description": ""
}


npm WARN calculator_dapp_client@1.0.0 No description0minkovacik/.npm/
npm WARN calculator_dapp_client@1.0.0 No repository field.

+ xhr2@0.1.4
+ bs58check@2.1.2
+ bip39@2.5.0
+ libsodium-wrappers@0.7.4
+ bignumber.js@8.0.2
updated 5 packages and audited 114 packages in 1.075s
found 0 vulnerabilities

--2019-01-24 21:39:32--  https://github.com/TezTech/eztz/raw/master/src/main.js
Resolving github.com (github.com)... 140.82.118.4, 140.82.118.3
Prip'ajanie k github.com (github.com)|140.82.118.4|

## Preparation of Test Account to be used by EZTZ
When interacting with tezos, it is required to have private/public key pair. This represents account, where we can move some tez and later to communicate with our smart contract.

Following code is equivalent to calling `tezos-client gen keys ...` command ([sample](calculator_dapp.ipynb#Testing-account-creation)). 

**WORKING NOTE: At the time of writing this guide, we were not able to use `tezos-client` generated keys in EZTZ, however keys generated by EZTZ work with tezos-client**.

Following call will generate private/public key, that we will later use for moving tez to this account and also to originate and call testing contract.

In [5]:
node <<EOM 2>/dev/null | tee keys.json
require('./eztz.js');

var node_address ='$TEZOS_NODE_URL';

(async() => {
    await eztz.library.sodium.ready;

    eztz.node.setProvider(node_address);

    var keys = eztz.crypto.generateKeysNoSeed()
    console.log(JSON.stringify(keys));
})();
EOM

export SECRET_KEY=unencrypted:$(jq -r ".sk" <keys.json)
export PUBLIC_KEY=$(jq -r ".pk" <keys.json)
export PUBLIC_KEY_HASH=$(jq -r ".pkh" <keys.json)

{"sk":"edskS2khcc94XMGXnkxCB8S7LX9ALMgpKyGsCj1RnLWAcKfzqdENpoikiGSHnuJdbzVq7qxUsTvGUwHVnZqbHjsjXBTZ4kYWBm","pk":"edpkuXpB6Cw8SjiYmqbNptNRi4gWignPh2m3ZUSWA7rdmB8bj9Pc13","pkh":"tz1TA8Y8yWrtd2iFjJwd1vq9RnMtLDA1mGUA"}


For convenience we will register given key pair with tezos-client. After registering the key we can validate, that generated public key and hash are the same as values provided by EZTZ.

In [6]:
tezos-client import secret key test_account $SECRET_KEY 
echo ----
echo Tezos-client registered keys:
tezos-client show address test_account -S
echo ----
echo EZTZ Generated keys:
jq <keys.json

Tezos address added: tz1TA8Y8yWrtd2iFjJwd1vq9RnMtLDA1mGUA
----
Tezos-client registered keys:
Hash: tz1TA8Y8yWrtd2iFjJwd1vq9RnMtLDA1mGUA
Public Key: edpkuXpB6Cw8SjiYmqbNptNRi4gWignPh2m3ZUSWA7rdmB8bj9Pc13
Secret Key: unencrypted:edskS2khcc94XMGXnkxCB8S7LX9ALMgpKyGsCj1RnLWAcKfzqdENpoikiGSHnuJdbzVq7qxUsTvGUwHVnZqbHjsjXBTZ4kYWBm
----
EZTZ Generated keys:
{
  "sk": "edskS2khcc94XMGXnkxCB8S7LX9ALMgpKyGsCj1RnLWAcKfzqdENpoikiGSHnuJdbzVq7qxUsTvGUwHVnZqbHjsjXBTZ4kYWBm",
  "pk": "edpkuXpB6Cw8SjiYmqbNptNRi4gWignPh2m3ZUSWA7rdmB8bj9Pc13",
  "pkh": "tz1TA8Y8yWrtd2iFjJwd1vq9RnMtLDA1mGUA"
}


We will move some tez to our newly create account. These will be later used for origination of smart contract and subsequent calls.

In [7]:
tezos-client transfer 1000 from bootstrap5 to test_account --burn-cap 0.257 > output.txt &
PROCESS_PID=$!

tezos-client bake for bootstrap1
wait $PROCESS_PID
cat output.txt

[2] 49796
Injected block BLGEPS5E5cb6
[2]+  Done                    tezos-client transfer 1000 from bootstrap5 to test_account --burn-cap 0.257 > output.txt
Node is bootstrapped, ready for injecting operations.
Estimated gas: 10100 units (will add 100 for safety)
Estimated storage: 257 bytes added (will add 20 for safety)
Operation successfully injected in the node.
Operation hash: ooLVnrfxxJP6fiySJrNMRjKPYnJX8G5P3FWZvPoahc27PN9Dw12
Waiting for the operation to be included...
Operation found in block: BLGEPS5E5cb6ivjpV4gzXvpsSnfvjv4LWgttfFDQD2wgQzvMVBw (pass: 3, offset: 0)
This sequence of operations was run:
  Manager signed operations:
    From: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
    Fee to the baker: ꜩ0.001275
    Expected counter: 1
    Gas limit: 10200
    Storage limit: 277 bytes
    Balance updates:
      tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv ........... -ꜩ0.001275
      fees(tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx,0) ... +ꜩ0.001275
    Transaction:
      Amount: ꜩ1000
      From:

After the transfer we can verify balance. Following code demonstrate checking of balance using EZTZ. EZTZ library is showing balance in mutez (1000000 mutez = 1 tez).

In [8]:
echo EZTZ
echo ----
node <<EOM 
require('./eztz.js');

var pkh = '$PUBLIC_KEY_HASH';
var node_address = 'http://$TEZOS_NODE_URL';

(async() => {
    await eztz.library.sodium.ready;

    eztz.node.setProvider(node_address);

    eztz.rpc.getBalance(pkh)
        .then(balance => {
            console.log("Balance is : " + balance + " mutez");    
    }
    );
})();
EOM
echo
echo tezos-client
echo ----
## call above is equivalent to 
tezos-client get balance for test_account

EZTZ
----
Balance is : 1000000000 mutez

tezos-client
----
1000 ꜩ


## Preparation of smart contract


In [9]:
#Sample preparation
mkdir -p ~/tezos-dev/calculator-dapp
cd ~/tezos-dev/calculator-dapp

/bin/cat <<EOM >Calculator.liq
[%%version 0.4]
  
type storage = int

let%init storage (init_storage : int) =
  init_storage
  
let%entry add (parameter : int) storage =
  ( ([] : operation list), storage + parameter )
  
let%entry subtract (parameter : int) storage =
  ( ([] : operation list), storage - parameter )
EOM

**WORKING NOTE: At the time of writing liquidity was not working with keys generated by EZTZ. For the guide we will continue using deployment using tezos-client.**

In [11]:
#liquidity \
#  --tezos-node $TEZOS_NODE_URL \
#  --source $PUBLIC_KEY_HASH \
#  --private-key $SECRET_KEY \
#  --fee 1tz \
#  --amount 3tz \
#  Calculator.liq \
#  --deploy '0' | tee contract_deploy_output.txt
  
#export CONTRACT_HASH=$(grep "New contract" contract_deploy_output.txt | cut -d" " -f3)
#export OPERATION_HASH=$(grep "New contract" contract_deploy_output.txt | cut -d" " -f7)

Response Error:
deploy: Bad private key


In [16]:
echo "Compiling Calculator contract to michelson"
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq 
echo ----
echo "Compiling Calculator initial storage"
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --init-storage 0
 
echo ------
echo Resulting michelson code:
cat Calculator.liq.tz

echo ------
echo Resulting storage:
cat Calculator.liq.init.tz

Compiling Calculator contract to michelson
Main contract Calculator
Storage initializer generated in "./calculator.liq.initializer.tz"
File "./calculator.liq.tz" generated
If tezos is compiled, you may want to typecheck with:
  tezos-client typecheck script ./calculator.liq.tz
----
Compiling Calculator initial storage
Main contract Calculator
Evaluated initial storage: 0
Constant initial storage generated in "calculator.liq.init.tz"
------
Resulting michelson code:
parameter (or :_entries (int %_Liq_entry_add) (int %_Liq_entry_subtract));
storage int;
code { DUP ;
       DIP { CDR @storage_slash_1 } ;
       CAR @parameter_slash_2 ;
       DUP @parameter ;
       IF_LEFT
         { RENAME @parameter_slash_3 ; DUUUP ; ADD ; NIL operation ; PAIR }
         { RENAME @parameter_slash_5 ; DUUUP ; SUB ; NIL operation ; PAIR } ;
       DIP { DROP ; DROP } };
------
Resulting storage:
0

## Development of Smart Contracts
At this point we have all prerequisites for preparing contract. 

Smart contracts for Tezos are implemented and using Michelson language. It's low-level stack-based language. We will refer to Michelson in other sections of this guide. We will implement sample contract using high level language Liquidity. Liquidity is provided with command line utility, that can be used to compile, deploy and manage the contracts developed using Liquidity language. Liquidity compiles into Michelson. All features, that are available in Michelson are possible in Liquidity as well.

Liquidity is a language heavily inspired by the OCaml language. Before starting working with Liquidity we advise 
to check some OCaml, F# or ReasonML tutorials to familiarize with language syntax (eg. https://try.ocamlpro.com).

To learn more about Liquidity, refer to [Liquidity website](http://www.liquidity-lang.org).

### Sample Calculator Contract
For beginning we will implement sample contract, that will be managing state and will enable add and subtract operations. Default operation will be add.

Smart contracts are software programs that binds the parties involved in a transaction to an agreement with out any third party being involved. The smart contract on the blockchain guarantees that nothing can be changed about the agreement without the consent of all the parties.

When you deploy smart contract into the blockchain it cannot be changed. It prescribes how it's internal storage should be ammended. When smart contract is called, all resulting storage changes are verified agains the smart contract code.

Each smart contract in liquidity consists of four parts
- version declaration (of liquidity)
- storage definition
- storage initialisation
- entry functions (at least method main is required. This method is called when contract is executed)

Following code shows structure of our initial version of contract.

```ocaml
[%%version 0.4]
(* We will store current state of calculator *) 
type storage = int

(* This function will initialise storage when originating contract *)
let%init storage (init_storage : int) =
  init_storage

let%entry add (parameter : int) storage =
  ( ([] : operation list), storage + parameter )
  
let%entry subtract (parameter : int) storage =
  ( ([] : operation list), storage - parameter )
```

Storage block is defining data structure, that will be representing internal state of the contract. For our sample we will use simple structure of type ```int```.

Function marked with `let%init` represents initialisation function for the storage. The function should return storage data structure (in this case int). Returned value would be used as a initial storage value.

Function main is default entry point for contract. Entry points are taking parameters, storage and returning tuple with list of operations, that should be applied after the contract is executed and storage value after execution of the contract. In our case we are not returning any operations, just incrementing value of internal storage.



#### Liquidity Development Tooling
Currently there are not many tools, that support development of liquidity contracts. As the language is very similar to OCaml, we suggest to use editors with OCaml syntax highlighting.

Other option is to use web [editor provided on liquidity website](http://www.liquidity-lang.org/edit/). This editor provides more advanced features like code formatting, code completion and comes also with lot of samples how to use liquidity.

### Contract Simulation and Deployment
#### Running Simulation of Contract using Liquidity
Tezos node can execute the script in Michelson and return result even without storage to blockchain. This can be used to simulate contract call during testing and development.

Liquidity can handle transpilation to Michelson and correct call of the script on our behalf. 

During the simulation, it's required to prepare storage structure before calling the contract entry point.
This can be achieved using following command.

In [ ]:
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --init-storage '1'

This command will call storage function (function where declaration starts with `let%init`) and pass arguments after `--init-storage` parameter. Result of the call is evaluated storage as it will be stored to blockchain, but still in Liquidity. Resulting storage in Michelson format will be stored in *.init.tz file.

Liquidity storage:

    Evaluated initial storage: *1*

Michelson storage:

In [ ]:
cat calculator.liq.init.tz

Now we have all required to run contract simulation using liquidity. For continuing it's important to understand tezos node functionality. Tezos node is important for managing of the blockchain, but it also acts as an interpreter for the Michelson contracts. Execution of the contract, can be tested even without storing results into blockchain.

Following command will execute liquidity script using tezos node.

In [ ]:
#in --run add 1 1 first parameter is name of entry point, 
# second represents input to the entry point, 
# third parameter represents storage
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --run add 1 1
 
echo ------

# call substract parameter:3 storage:10
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --run subtract 3 10


#### Direct Liquidity Deployment
***As of now both liquidity and [tezos-client deployment](#Running-of-Contract-Using-Tezos-Client) are working. Consider these two as alternatives. Tezos-client is following changes is in sync with tezos sources, so it is guaranteed to work.***

To deploy the contract to tezos, we will use following command. As parameters we are passing source account and it's private key. In attribute deploy we will pass parameters required for storage initialisation.

Before continuing we need to bake the block with operation.

In [ ]:
tezos-client bake for bootstrap1 2>/dev/null

To review storage of the contract we can use liquidity program with command --get-storage. As a parameter we are passing hash of the function.

In [ ]:
liquidity \
 --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --get-storage $CONTRACT_HASH

Now we can start working with the contract.

In [ ]:
liquidity \
 --tezos-node $TEZOS_NODE_URL \
 --private-key $TZ_SECRET \
 Calculator.liq \
 --call $CONTRACT_HASH add '10'

After executing the command we still don't see results of our operation in storage. To see changes, we need to bake the block first. This is specific of sandbox environment. In Alphanet, baking will be handled by dedicated bakers.

In [ ]:
echo "Storage before bake"
liquidity \
 --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --get-storage $CONTRACT_HASH
 
tezos-client bake for bootstrap1 2>/dev/null

echo "Storage after the block is baked"
liquidity \
 --tezos-node localhost:18731 \
 Calculator.liq \
 --get-storage $CONTRACT_HASH

### Running of Contract Using Tezos Client
For this section we will use `liquidity` as a compiler only. When calling the command with the liquidity program, it will compile the code into the Michelson program. 


#### Compilation of Contract to Michelson
Liquidity is hiding lot of complexity, however in some scenarios it is required to use Michelson. Following sample is showing how to use liquidity in order to create michelson script.

When liquidity script is compiled, storage initialization and contract entry points are provided separately. 

Liquidity is providing compilation also for method invocations. This way it simplifies creation of required michelson statements.

In [ ]:
# Evaluation of the storage structure - calling of storage function and resulting storage structure is outputed
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --init-storage '1'

echo ------
echo add 1
# Compilation of function call - calling of add entry point with argument "1"
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --data add '1'
 
echo ------
echo subtract 4
# Compilation of function call - calling of subtract entry point with argument "4"
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --data subtract '4'

Now it's possible to simulate the contract directly in Michelson.

In [ ]:
echo "Calling add 10, initial storage 0 (calculator.liq.init.tz)"
tezos-client run script calculator.liq.tz on storage $(<calculator.liq.init.tz) and input '(Left 10)'

echo "Calling subtract 5, initial storage 3"
tezos-client run script calculator.liq.tz on storage 3 and input '(Right 5)'

#### Deployment of Contract using Tezos Client
For deployment we need to create originated account. This can be achieved using `tezos-client originate contract...` call. During the origination it's required to provide our private/secret key ([command to retrieve](#Testing-account-creation)).

In [ ]:
# We are creating new originated contract and registring it's alias Calculator with our tezos-client.
tezos-client originate contract Calculator \
 for test_account \
 transferring 1 from test_account \
 running calculator.liq.tz \
 --init $(<calculator.liq.init.tz) \
 --burn-cap 1 > contract_output.txt &

PROCESS_PID=$!

Operation is not reflected in tezos blockchain until new block is baked. 

In [ ]:
tezos-client bake for bootstrap1

wait $PROCESS_PID
cat contract_output.txt

`tezos-client` registers alias for newly created contract. The alias is matching hash outputed in the originate call above.


In [ ]:
CONTRACT_HASH=$(grep "New contract KT" contract_output.txt | cut -d" " -f3)
echo "Contract Hash $CONTRACT_HASH"
echo ------
echo "Calculator listed among aliases"
tezos-client list known contracts

### Call of contract using tezos-client
Calling of smart contract is the same as transfering balance. When calling smart contract we are providing additional arguments, that should be in Michelson - refer to [Michelson compilation](#Compilation-of-Contract-to-Michelson) for sample how to get to Michelson function call for liquidity function.

In [ ]:
#Calling our contract, entry point add (using parameter generated by liquidity compiled function call)
tezos-client \
 transfer 0 from test_account to $CONTRACT_HASH --arg "(Left 10)" > output.txt &
PROCESS_PID=$!

#Baking manually
tezos-client bake for bootstrap1  #baking the block
wait $PROCESS_PID
cat output.txt

We can verify contract call by querying storage in blockchain directly.

In [ ]:
tezos-client get script storage for Calculator
# curl -s http://$TEZOS_NODE_URL/chains/main/blocks/head/context/contracts/$CONTRACT_HASH/storage | jq

Alternatively we can check the block in which the operation was included.

In [ ]:
curl -s http://$TEZOS_NODE_URL/chains/main/blocks/head | jq

## Implementation of application front-end
For the implementation of front-end we will use Eztz library (https://github.com/TezTech/eztz). This library is implementing RPC communication with Tezos and also provides functionality for key generation, signing, verification, and contract interaction.

For the sample we will fetch library from github (At the time of writing Eztz does not provide NPM package).



In [ ]:
cd ~/tezos-dev/calculator-dapp

#this will create default npm module
npm init -y

#this will download eztz library for CLI
wget https://github.com/TezTech/eztz/raw/master/dist/eztz.cli.js

#this will download eztz dependencies required for CLI
npm install --save xmlhttprequest
npm install --save bs58check
npm install --save libsodium-wrappers
npm install --save bip39

To be able to interact with our smart contract, we need to have account - this is represented by private/public key pair. For the demo we will create account in JS application - this is the activity we did earlier as [Testing Account Creation](#Testing-account-creation).

    TBD - continue with content

## Sandbox Clean-up

Command to terminate sandbox blockchain running in the background

In [ ]:
kill $(<$TEZOS_HOME/tezos_sandbox_pid.txt)